# Tutorial: Nash-Sutcliffe Efficiency (NSE) Calculation
In this tutorial, we will explore the Nash-Sutcliffe Efficiency (NSE) calculation function `nse`. NSE is a widely used metric in hydrology and other fields to evaluate the performance of a model by comparing its predictions to observed data.

## Introduction

The Nash-Sutcliffe Efficiency (NSE) is defined as:

$NSE = 1 - \frac{\sum_{i=1}^{n}(f_i - o_i)^2}{\sum_{i=1}^{n}(o_i - \bar{o})^2}$

Where:
- $f_i$ is the forecast or predicted value
- $o_i$ is the observed value
- $\bar{o}$ is the mean of the observed values
- $n$ is the number of data points

A perfect model has an NSE value of 1, while a model performing as poorly as the mean of the observed data has an NSE value of 0 or less.

## Using the `nse` Function

Let's start by importing the `nse` function from our module and exploring its usage with different types of input data.

In [29]:
from scores.continuous import nse
import numpy as np
import xarray as xr
import pandas as pd

np.random.seed(0)  # set the seed to make notebook reproducible

**Example 1**: NumPy Arrays

In [27]:
fcst_np = np.array([3, 4, 5, 6, 7])
obs_np = np.array([2, 3, 4, 5, 6])
nse_np = nse(fcst_np, obs_np)
print("NSE for NumPy arrays:", nse_np)

NSE for NumPy arrays: <xarray.DataArray ()>
array(0.5)


**Example 2**: Pandas Series

In [30]:
fcst_series = pd.Series([3, 4, 5, 6, 7])
obs_series = pd.Series([2, 3, 4, 5, 6])
nse_series = nse(fcst_series, obs_series)
print("NSE for Pandas Series:", nse_series)

NSE for Pandas Series: <xarray.DataArray ()>
array(0.5)


**Example 3**: Xarray DataArray

In [31]:
fcst_xr = xr.DataArray([3, 4, 5, 6, 7])
obs_xr = xr.DataArray([2, 3, 4, 5, 6])
nse_xr = nse(fcst_xr, obs_xr)
print("NSE for Xarray DataArray:", nse_xr)

NSE for Xarray DataArray: <xarray.DataArray ()>
array(0.5)


**Example 4**: Lists

In [32]:
fcst_list = [3, 4, 5, 6, 7]
obs_list = [2, 3, 4, 5, 6]
nse_list = nse(fcst_list, obs_list)
print("NSE for lists:", nse_list)

NSE for lists: <xarray.DataArray ()>
array(0.5)


**Example 5**: Mixed Types

In [33]:
fcst_mix = np.array([3, 4, 5, 6, 7])
obs_mix = pd.Series([2, 3, 4, 5, 6])
nse_mix = nse(fcst_mix, obs_mix)
print("NSE for mixed types:", nse_mix)

NSE for mixed types: <xarray.DataArray ()>
array(0.5)


**Example 6**: Large Xarray DataArrays

In [34]:
fcst_large = xr.DataArray(
    data=np.random.random_sample((1000, 1000)) * 360, 
    dims=["space", "time"], 
    coords=[np.arange(0, 1000),  np.arange(0, 1000)]
)
obs_large = xr.DataArray(
    data=np.random.random_sample((1000, 1000)) * 360, 
    dims=["space", "time"], 
    coords=[np.arange(0, 1000),  np.arange(0, 1000)]
)
nse_large = nse(fcst_large, obs_large)
print("NSE for large Xarray DataArrays:", nse_large)


NSE for large Xarray DataArrays: <xarray.DataArray ()>
array(-0.9995806)


**Example 7**: Angular and array

In [35]:
fcst = np.array([0, 90, 180, 270, 360])
obs = np.array([0, 90, 180, 270, 360])
nse_anular = nse(fcst, obs,angular=True)
print("NSE for angular types:", nse_anular )

NSE for angular types: <xarray.DataArray ()>
array(1.)


**Example 8**: Weight and array

In [37]:

fcst = np.array([3, 4, 5, 6, 7])
obs = np.array([2, 3, 4, 5, 6])
weights = np.array([1, 2, 3, 2, 1])
nse_weights=nse(fcst, obs, weights=weights)
print("NSE with weights types:", nse_weights )


NSE with weights types: <xarray.DataArray ()>
array(0.5)
